In [1]:
from data_processor import DataProcessor
import torch 
import torch.nn as nn
from torch.optim import Adam

In [2]:
processor = DataProcessor("../data/Train.csv", test = False, minimal = True)
x_train, x_valid, y_train, y_valid = processor.get_numpy_data(True, True, True, False, True)

In [3]:
x_train = torch.tensor(x_train).float()
x_valid = torch.tensor(x_valid).float()
y_train = torch.tensor(y_train).float()
y_valid = torch.tensor(y_valid).float()

In [4]:
batch_size = 1024
n_epochs = 1000000

In [5]:
linear_regressor = nn.Linear(in_features = len(x_train[0]), out_features = 1)


The loss after 1000 epoch is:  1021640.0625
The loss after 1000 epoch is:  692014.375
The loss after 1000 epoch is:  688944.5625
The loss after 1000 epoch is:  686233.6875
The loss after 1000 epoch is:  683843.1875
The loss after 1000 epoch is:  681717.125
The loss after 1000 epoch is:  679812.9375
The loss after 1000 epoch is:  678095.0
The loss after 1000 epoch is:  676539.0
The loss after 1000 epoch is:  675124.9375
The loss after 1000 epoch is:  673835.0625
The loss after 1000 epoch is:  672656.0625
The loss after 1000 epoch is:  671576.875
The loss after 1000 epoch is:  670586.0625
The loss after 1000 epoch is:  669676.0
The loss after 1000 epoch is:  668838.875
The loss after 1000 epoch is:  668068.8125
The loss after 1000 epoch is:  667358.25
The loss after 1000 epoch is:  666702.25
The loss after 1000 epoch is:  666096.375
The loss after 1000 epoch is:  665536.125
The loss after 1000 epoch is:  665017.5625
The loss after 1000 epoch is:  664537.625
The loss after 1000 epoch is: 

KeyboardInterrupt: 

In [6]:
def train(model): 
    cirterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr = .05)
    for i in range (n_epochs): 
        for b in range(0, len(x_train), batch_size):
            predictions = model(x_train[b:b+batch_size])
            loss = cirterion(predictions, y_train[b:b+batch_size])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i % 1000 ==0: 
            print("The loss after 1000 epoch is: ", loss.item())

In [8]:
neural_regressor = nn.Sequential(
nn.Linear(in_features = len(x_train[0]), out_features = 15),
nn.ReLU(), 
nn.Linear(in_features = 15, out_features = 7), 
nn.ReLU(),
nn.Linear(in_features = 7, out_features = 1))


In [9]:
train(neural_regressor)

The loss after 1000 epoch is:  1104506.125
The loss after 1000 epoch is:  665004.0
The loss after 1000 epoch is:  658509.375
The loss after 1000 epoch is:  645466.8125


KeyboardInterrupt: 